# Titanic - Machine Learning from Disaster

<img src="https://www.cnnbrasil.com.br/wp-content/uploads/sites/12/2023/06/titanic_navio_1912-e1687528104691.jpg?w=876&h=484&crop=1" width=900>

- Vamos empregar os dados atualmente acessíveis no Kaggle. [dados disponíveis no Kaggle](https://www.kaggle.com/competitions/titanic)
    - Trata-se de um conjunto de dados destinado a uma competição
    - A avaliação do desempenho decorre com base na acurácia:

  *Sua pontuação reflete a porcentagem de passageiros que você é capaz de prever com precisão. Esse índice é conhecido como acurácia.*

### Importando novamente as bases e fazendo o tratamento dos dados


In [ ]:
# Importando o pandas
import pandas as pd

In [ ]:
# Visualizando a base de treino
train = pd.read_csv('train.csv')
train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [ ]:
# Visualizando a base de teste
test = pd.read_csv('test.csv')
test.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


- **Fazendo o mesmo tratamento inicial que fizemos na aula anterior**

In [ ]:
# Eliminando as colunas com elevada cardinalidade
train = train.drop(['Name','Ticket','Cabin'],axis=1)
test = test.drop(['Name','Ticket','Cabin'],axis=1)

In [ ]:
# Usando a média para substituir valores nulos na coluna de idade
train.loc[train.Age.isnull(),'Age'] = train.Age.mean()
test.loc[test.Age.isnull(),'Age'] = test.Age.mean()

In [ ]:
# Tratando a coluna Embarked da base de treino usando a moda
train.loc[train.Embarked.isnull(),'Embarked'] = train.Embarked.mode()[0]

In [ ]:
# E também a coluna Fare da base de teste usando a média
test.loc[test.Fare.isnull(),'Fare'] = test.Fare.mean()

### Podemos agora entender as colunas de texto
- Vamos agora **adicionar um novo tratamento das colunas de texto**

In [ ]:
# Verificando as colunas de texto na base de treino
train.columns[train.dtypes == 'object']

Index(['Sex', 'Embarked'], dtype='object')

In [ ]:
# Verificando os valores na coluna Sex
train.Sex.value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [ ]:
# e na coluna Embarked
train.Embarked.value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

- Para abordar a coluna "**Sex**", é possível criar uma nova coluna denominada "Male_Check", a qual será atribuída o valor 1 para o gênero masculino e 0 para o gênero feminino.

In [ ]:
# Usando uma lambda function para fazer esse tratamento
train['MaleCheck'] = train.Sex.apply(lambda x: 1 if x == 'male' else 0)

In [ ]:
# Verificando os valores
train[['Sex','MaleCheck']].value_counts()

Sex     MaleCheck
male    1            577
female  0            314
dtype: int64

- O próximo passo é fazer o mesmo para a base de teste

In [ ]:
# Usando uma lambda function para fazer esse tratamento
test['MaleCheck'] = test.Sex.apply(lambda x: 1 if x == 'male' else 0)

In [ ]:
# Verificando os valores
test[['Sex','MaleCheck']].value_counts()

Sex     MaleCheck
male    1            266
female  0            152
dtype: int64

- Para lidar com a coluna "Embarked", empregaremos o método **[OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)** o qual resultará na criação de novas colunas correspondentes a cada um dos rótulos presentes na coluna original.

In [ ]:
# Importando o OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# Criando o encoder
ohe = OneHotEncoder(handle_unknown='ignore',dtype='int32')

In [ ]:
# Fazendo o fit com os dados
ohe = ohe.fit(train[['Embarked']])

In [ ]:
# Fazendo a transformação
ohe.transform(train[['Embarked']]).toarray()

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       ...,
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0]], dtype=int32)

In [ ]:
# Transformando esse resultado em um DataFrame
ohe_df = pd.DataFrame(ohe.transform(train[['Embarked']]).toarray(),columns=ohe.get_feature_names_out())
ohe_df.head()

,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1


In [ ]:
# Podemos agora adicionar essa coluna na nossa base de treino
train = pd.concat([train,ohe_df],axis=1)

In [ ]:
# Verificando os valores
train[['Embarked','Embarked_C','Embarked_Q','Embarked_S']].value_counts()

Embarked  Embarked_C  Embarked_Q  Embarked_S
S         0           0           1             646
C         1           0           0             168
Q         0           1           0              77
dtype: int64

- Agora vamos **fazer o mesmo para a base de teste usando o encoder ohe que criamos acima**

In [ ]:
# Transformando esse resultado em um DataFrame
ohe_df = pd.DataFrame(ohe.transform(test[['Embarked']]).toarray(),columns=ohe.get_feature_names_out())

In [ ]:
# Adicionando o resultado na base de teste
test = pd.concat([test,ohe_df],axis=1)

In [ ]:
# Verificando também os valores
test[['Embarked','Embarked_C','Embarked_Q','Embarked_S']].value_counts()

Embarked  Embarked_C  Embarked_Q  Embarked_S
S         0           0           1             270
C         1           0           0             102
Q         0           1           0              46
dtype: int64

### Usando essa nova base no modelo

In [ ]:
# Visualizando a base
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,MaleCheck,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,male,22.0,1,0,7.2500,S,1,0,0,1
1,2,1,1,female,38.0,1,0,71.2833,C,0,1,0,0
2,3,1,3,female,26.0,0,0,7.9250,S,0,0,0,1
3,4,1,1,female,35.0,1,0,53.1000,S,0,0,0,1
4,5,0,3,male,35.0,0,0,8.0500,S,1,0,0,1


In [ ]:
# Podemos então apagar essas duas colunas que já tratamos
train = train.drop(['Sex','Embarked'],axis=1)
test = test.drop(['Sex','Embarked'],axis=1)

- Podemos selecionar os mesmos modelos que vimos anteriormente (consulte arquivo da **[parte 1](https://github.com/lucaslealx/Titanic/blob/main/Parte1.ipynb)**)
    - **Árvore de classificação**
        - https://scikit-learn.org/stable/modules/tree.html#classification
    - **Classificação dos vizinhos mais próximos**
        - https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier
    - **Regressão Logística**
        - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression
- Antes de usar os algoritmos, precisamos separar a base de treino em **treino e validação**
    - Vamos fazer isso utilizando o **train_test_split**
        - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
# Importando o train_test_split
from sklearn.model_selection import train_test_split

In [ ]:
# Separando a base de treino em X e y
X = train.drop(['PassengerId','Survived'],axis=1)
y = train.Survived

In [ ]:
# Separando em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

- Para a **árvore de classificação**

In [ ]:
# Fazendo a importação
from sklearn import tree

In [ ]:
# Criando o classificador
clf_ac = tree.DecisionTreeClassifier(random_state=42)

In [ ]:
# Fazendo o fit com os dados
clf_ac = clf_ac.fit(X_train,y_train)

In [ ]:
# Fazendo a previsão
y_pred_ac = clf_ac.predict(X_val)

- Para o **KNeighborsClassifier**

In [ ]:
# Importando
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# Criando o classificador
clf_knn = KNeighborsClassifier(n_neighbors=3)

In [ ]:
# Fazendo o fit com os dados
clf_knn = clf_knn.fit(X_train,y_train)

In [ ]:
# Fazendo a previsão
y_pred_knn = clf_knn.predict(X_val)

- E para a **Regressão Logística**

In [ ]:
# Importando
from sklearn.linear_model import LogisticRegression

In [ ]:
# Criando o classificador
clf_rl = LogisticRegression(random_state=42,max_iter=1000)

In [ ]:
# Fazendo o fit com os dados
clf_rl = clf_rl.fit(X_train,y_train)

In [ ]:
# Fazendo a previsão
y_pred_rl = clf_rl.predict(X_val)

- Vamos novamente **avaliar os modelos**
    - Acurácia (método de avaliação usado na competição):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
    - Matriz de confusão (ajuda a visualizar a distribuição dos erros):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

- Avaliando a **acurácia**

In [ ]:
# Importando
from sklearn.metrics import accuracy_score

In [ ]:
# Para a árvore
accuracy_score(y_val, y_pred_ac)

0.7491525423728813

In [ ]:
# Para o knn
accuracy_score(y_val, y_pred_knn)

0.7152542372881356

In [ ]:
# Para a regressão logística
accuracy_score(y_val, y_pred_rl)

0.8169491525423729

- Avaliando a **matriz de confusão**

In [ ]:
# Importando
from sklearn.metrics import confusion_matrix

In [ ]:
# Para a árvore
confusion_matrix(y_val, y_pred_ac)

array([[138,  37],
       [ 37,  83]])

In [ ]:
# Para o knn
confusion_matrix(y_val, y_pred_knn)

array([[147,  28],
       [ 56,  64]])

In [ ]:
# Para a regressão logística
confusion_matrix(y_val, y_pred_rl)

array([[153,  22],
       [ 32,  88]])

### Fazendo a previsão para os dados de teste
- Vamos usar o modelo com melhor precisão para fazer o predict na base de teste

In [ ]:
# Visualizando o X_train
X_train.head(3)

,Pclass,Age,SibSp,Parch,Fare,MaleCheck,Embarked_C,Embarked_Q,Embarked_S
6,1,54.000000,0,0,51.8625,1,0,0,1
718,3,29.699118,0,0,15.5000,1,0,1,0
685,2,25.000000,1,2,41.5792,1,1,0,0


In [ ]:
# Visualizando a base de teste
test.head(3)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,MaleCheck,Embarked_C,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,7.8292,1,0,1,0
1,893,3,47.0,1,0,7.0000,0,0,0,1
2,894,2,62.0,0,0,9.6875,1,0,1,0


In [ ]:
# Para a base de teste ser igual a base de treino, precisamos eliminar a coluna de id
X_test = test.drop('PassengerId',axis=1)

In [ ]:
# Utilizando a regressão logística na base de teste
y_pred = clf_rl.predict(X_test)

In [ ]:
# Criando uma nova coluna com a previsão na base de teste
test['Survived'] = y_pred

In [ ]:
# Selecionando apenas a coluna de Id e Survived para fazer o envio
base_envio = test[['PassengerId','Survived']]

In [ ]:
# Exportando para um csv
base_envio.to_csv('resultado2.csv',index=False)